# String alignment using dynamic programming

In [1]:
import numpy as np
import random

Alphabet of the strings we consider for alignment, for this example we use nucliotides that encode DNA.

In [2]:
alphabet = ['A', 'C', 'G', 'T']

Scoring matrix, substituting an `A` by a `C` or `T` yields a penalty of -1, while substituing an `A` by a `G` yields a reward of 1, and matching an `A` yields 2.

In [3]:
scoring_matrix = np.array(
    [[ 2, -1,  1, -1],
     [-1,  2, -1,  1],
     [ 1, -1,  2, -1],
     [-1,  1, -1,  2]]
)

Alignments can introduce gaps, and they come with an associated score as well.

In [15]:
gap_score = -2

The following function generates a function to generate random strings for the given alphabet.

In [4]:
def random_string_generator(alphabet):
    def random_string(n):
        return ''.join(random.choices(alphabet, k=n))
    return random_string

Create a function to generate DNA fragments of a given length.

In [5]:
random_dna = random_string_generator(alphabet)

Seed the random number generator for reproducible results.

In [6]:
random.seed(1234)

Test the function for a few strings:

In [7]:
for i in range(3, 21):
    print(random_dna(i//3))

T
C
A
TT
GG
AT
AAT
CGG
AAA
ACTA
GCGA
GCGG
CCATC
GAGAA
CCAGA
GATTTA
CACTAA
TATACC


Implement the alignment algorithm as a class so that the alphabet, the scoring matrix and the gap score can be initialized once for alignment of many string pairs.

In [11]:
class Aligner:
    
    def __init__(self, alphabet, scoring_matrix, gap_score):
        self._idx = {char: index for index, char in enumerate(alphabet)}
        self._scoring_matrix = scoring_matrix
        self._gap_score = gap_score
 
    def _init(self, str1, str2):
        self._str1 = 'X' + str1
        self._str2 = 'X' + str2
        self._dist = np.empty((1 + len(str1), 1 + len(str2)), np.int)
        self._dist[0, 0] = 0
        for i in range(1, self._dist.shape[0]):
            self._dist[i, 0] = self._dist[i - 1, 0] + self._gap_score
        for j in range(1, self._dist.shape[1]):
            self._dist[0, j] = self._dist[0, j - 1] + self._gap_score
        
    def _compute_edit_distance(self):
        # from IPython.core.debugger import Tracer; Tracer()()
        for i in range(1, self._dist.shape[0]):
            for j in range(1, self._dist.shape[1]):
                self._dist[i, j] = self._distance(i, j)
    
    def _distance(self, i, j):
        idx1 = self._idx[self._str1[i]]
        idx2 = self._idx[self._str2[j]]
        match = self._dist[i - 1, j - 1] + self._scoring_matrix[idx1, idx2]
        gap1 = self._dist[i, j - 1] + self._gap_score
        gap2 = self._dist[i - 1, j] + self._gap_score
        return max((match, gap1, gap2,))

    def align(self, str1, str2):
        self._init(str1, str2)
        self._compute_edit_distance()
        a1, a2 = '', ''
        i, j = self._dist.shape[0] - 1, self._dist.shape[1] - 1
        while i > 0 and j  > 0:
            idx1 = self._idx[self._str1[i]]
            idx2 = self._idx[self._str2[j]]
            match = self._dist[i - 1, j - 1] + self._scoring_matrix[idx1, idx2]
            if self._dist[i, j] == match:
                a1 = self._str1[i] + a1
                a2 = self._str2[j] + a2
                i, j = i - 1, j - 1
            elif  self._dist[i, j] == self._dist[i - 1, j] + self._gap_score:
                a1 = self._str1[i] + a1
                a2 = '_' + a2
                i -= 1
            elif self._dist[i, j] == self._dist[i, j - 1] + self._gap_score:
                a1 = '_' + a1
                a2 = self._str2[j] + a2
                j -= 1
        if i > 0:
            while i > 0:
                a1 = self._str1[i] + a1
                a2 = '_' + a2
                i -= 1
        elif j > 0:
            while j > 0:
                a1 = '_' + a1
                a2 = self._str2[j] + a2
                j -= 1
        return a1, a2

    @property
    def index(self):
        return self._idx
            
    @property
    def string1(self):
        return self._str1[1:]
    
    @property
    def string2(self):
        return self._str2[1:]
    
    @property
    def distance_matrix(self):
        return np.copy(self._dist)

Test the implementation on the example of http://www.biorecipes.com/DynProgBasic/code.html

In [17]:
alignment = Aligner(alphabet=alphabet, scoring_matrix=scoring_matrix, gap_score=gap_score)
a1, a2 = alignment.align('CCTAAG', 'ACGGTAG',)
print(a1)
print(a2)

CCTA_AG
ACGGTAG


The result is tested against the expected result:

In [24]:
assert(a1 == 'CCTA_AG')
assert(a2 == 'ACGGTAG')

In [18]:
alignment.distance_matrix

array([[  0,  -2,  -4,  -6,  -8, -10, -12, -14],
       [ -2,  -1,   0,  -2,  -4,  -6,  -8, -10],
       [ -4,  -3,   1,  -1,  -3,  -3,  -5,  -7],
       [ -6,  -5,  -1,   0,  -2,  -1,  -3,  -5],
       [ -8,  -4,  -3,   0,   1,  -1,   1,  -1],
       [-10,  -6,  -5,  -2,   1,   0,   1,   2],
       [-12,  -8,  -7,  -3,   0,   0,   1,   3]])

To test implementations of new algorithms or in other programming languages, a test set is quite useful.  For each example, two DNA strings are generated.  The first will have a length between `min_length` and `max_length`, and the second string's length will deviated from that by a number between `min_dev` and `max_dev` (`min_legnth` can be negative, if so, it should be choosen such that `min_length` + `_min_dev` > 0.

In [33]:
random.seed(1234)
file_name = 'alignment_data.txt'
min_length, max_length = 6, 12
min_dev, max_dev = -5, 5
nr_samples = 10
aligner = Aligner(alphabet, scoring_matrix, gap_score)
data = []
for sample_nr in range(nr_samples):
    data_str = ''
    n1 = random.randint(min_length, max_length)
    n2 = n1 + random.randint(min_dev, max_dev)
    str1, str2 = random_dna(n1), random_dna(n2)
    data_str += f'{str1} {str2}'
    aligned1, aligned2 = aligner.align(str1, str2)
    data_str += f' {aligned1} {aligned2}'
    data.append(data_str)
data_str = '\n'.join(data)
print(data_str)
with open(file_name, 'w') as file:
    print(data_str, file=file)

AATTAGACATAG CCAGTGTAGTAGGA __AAT_TAGACATAG CCAGTGTAG_TAGGA
ACTCGATGTACG GAACAAACAGGCCGA _ACTCGATGTA_CG_ GAACAAACAGGCCGA
CTTCGTCG CATTG CTTCGTCG ___CATTG
GGCCTCT TATCTCAAG GGCCTC__T TATCTCAAG
TCATGTC ACACAGATGCT _T_C__ATGTC ACACAGATGCT
CACACG TGTCTGGAGC CA_C__ACG_ TGTCTGGAGC
GCTGCAGTTACA GGTAACGATCTT GCT_GCAGTTACA GGTAACGATC_TT
TCGTTACCTC ACAGGATCTACACCG TC__G_T_TAC_CTC ACAGGATCTACACCG
CCCCGAGTC GGTGGA CCCCGAGTC _GGTGGA__
AATGTTTC GCTCATCGGC AAT_GTT_TC GCTCATCGGC
